In [ ]:
import numpy  as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns


import nltk

import warnings
warnings.filterwarnings('ignore')

import nltk
nltk.download('punkt')

import regex as re

from sklearn.preprocessing import LabelEncoder

from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords


import spacy
from nltk.stem import WordNetLemmatizer



# Scipy
import scipy


# Train-test split and cross validation
from sklearn.model_selection import train_test_split, ParameterGrid

# Classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB        
# from sklearn.naive_bayes import MultinomialNB     
from sklearn.naive_bayes import BernoulliNB     
from sklearn.naive_bayes import CategoricalNB  

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
